In [1]:
import pandas as pd
import yfinance as yf

from datetime import datetime, date

In [2]:
def grab_symbols():
    # Grab S&P Symbols from Wikipedia or local HTML File
    # wiki_url = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'
    # wiki_url = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies#S&P_500_component_stocks'
    tickers = pd.read_html('./tickers.html')[0]
    tickers = tickers.Symbol.to_list()
    tickers = [i.replace('.','-') for i in tickers]
    return tickers 

print('Grab symbols function defined...')

Grab symbols function defined...


In [3]:
def yahoo_prices(tickers, current_day):
    # Return a DataFrame with dates, symbols and prices
    if not isinstance(tickers,list):
        return None

    years_back = 5
    end_dt = datetime.strptime(current_day, '%Y-%m-%d').date()
    start_dt = date(end_dt.year - years_back, end_dt.month, end_dt.day)

    data = yf.download(tickers, start=start_dt, end=end_dt)
    data = data.loc[(slice(None)),(slice(None),slice(None))].copy()
    data = data.stack()
    data = data.reset_index()
    data.rename(columns={'level_1': 'Symbol'}, inplace=True)
    data.set_index('Date', inplace=True)
    return data

print('Yahoo_prices function defined...') 

Yahoo_prices function defined...


In [14]:
def detect_engulfing(df):
    cond_1 = df.shift(1).Close < df.shift(1).Open
    cond_2 = df.Close > df.Open
    cond_3 = df.Open < df.shift(1).Close
    cond_4 = df.Close > df.shift(1).Open
    df['Engulfing'] = cond_1 & cond_2 & cond_3 & cond_4
    return df 

print('detect_engulfing function defined')

detect_engulfing function defined


In [22]:
def trades(data, cash=10000, cash_percentage=0.15, max_hold_days=20, stop_percentage=0.90, target_percentage=1.30):
    #cash = 10000
    #cash_percentage = 0.15
    #max_hold_days = 20
    #stop_percentage = 0.90
    #target_percentage = 1.30
    in_position = False
    trade_id = 0
    cumul_profit = 0
    all_trades = []
    for index, row in data[:].iterrows():
        if not in_position and row.Buy:
            buy_date        = index
            buy_price       = row.Open
            stop_loss       = buy_price * stop_percentage
            target_price    = buy_price * target_percentage
            amount_invested = cash * cash_percentage
            shares          = int(amount_invested / buy_price)
            buy_amount      = shares * buy_price
            cash            -= buy_amount
            in_position = True
            sell_action = False
            trade = [trade_id, buy_date, buy_price, stop_loss, target_price, amount_invested, shares, buy_amount]
            #print(f'{buy_date} -- BUY___[ {shares} * ${row.Open:0.2f} = ${buy_amount:.2f}] [ ${stop_loss:0.2f}, ${target_price:0.2f} ]')
        if in_position:
            if row.Low < stop_loss:
                sell_price  = row.Low
                sell_action = True
                sell_type   = 'Stop__'
            if row.High > target_price:
                sell_price = row.High
                sell_action = True
                sell_type   = 'Target'
            time_in_position = index - buy_date
            if time_in_position.days >= max_hold_days:
                sell_price = row.Close
                sell_action = True
                sell_type   = 'Close_'

            if sell_action:
                sell_date   = index
                sold_amount = shares * sell_price
                cash += sold_amount
                profit = sold_amount - buy_amount
                #print(f'{sell_date} -- {sell_type}[ {shares} * ${row.Low:0.2f} = ${sell_amount:0.2f}] [Profit: {profit:0.2f} ]')
                in_position = False
                sell_action = False
                sell_info = [sell_date, sell_price, shares, sold_amount, sell_type, profit, cash]
                trade.extend(sell_info)
                cumul_profit += profit
                trade_id += 1
                all_trades.append(trade)

    return all_trades

    print('trades function defined')


In [6]:
my_symbols = grab_symbols()
tsla = my_symbols.index('TSLA')
(tsla, my_symbols[tsla])


(445, 'TSLA')

In [7]:
data = yahoo_prices(my_symbols[440:450], '2023-02-14')

[*********************100%***********************]  10 of 10 completed


In [8]:
tsla_df = data.loc[data.Symbol == 'TSLA'].copy()
tsla_df = detect_engulfing(tsla_df)
tsla_df.loc[tsla_df.Engulfing == True]
tsla_df['Buy'] = tsla_df.shift(1).Engulfing
tsla_df.loc[tsla_df.Engulfing | tsla_df.Buy]

,Symbol,Adj Close,Close,High,Low,Open,Volume,Engulfing,Buy
Date,,,,,,,,,
2018-03-21 00:00:00-04:00,TSLA,21.101999,21.101999,21.496000,20.679333,20.683332,89376000,True,False
2018-03-22 00:00:00-04:00,TSLA,20.606667,20.606667,21.254667,20.545334,20.926001,74097000,False,True
2018-03-29 00:00:00-04:00,TSLA,17.742001,17.742001,18.063999,16.547333,17.099333,227560500,True,False
2018-04-02 00:00:00-04:00,TSLA,16.832001,16.832001,17.355333,16.306000,17.084000,241710000,False,True
2018-04-04 00:00:00-04:00,TSLA,19.129333,19.129333,19.224667,16.799999,16.851999,298450500,True,False
...,...,...,...,...,...,...,...,...,...
2022-07-14 00:00:00-04:00,TSLA,238.313339,238.313339,238.653336,229.333328,234.896667,78557400,False,True
2022-10-21 00:00:00-04:00,TSLA,214.440002,214.440002,214.660004,203.800003,206.419998,75713800,True,False
2022-10-24 00:00:00-04:00,TSLA,211.250000,211.250000,213.500000,198.589996,205.820007,100446800,False,True


In [24]:
trade_log = trades(tsla_df, cash=10000, cash_percentage=0.5, max_hold_days=5, stop_percentage=0.90, target_percentage=1.30)
trades_df = pd.DataFrame(trade_log)
trades_df.columns = ['trade_id', 'buy_date', 'buy_price', 'stop_loss', 'target_price', 'amount_invested', 'shares', 'buy_amount', 'sell_date', 'sell_price', 'shares', 'sell_amount', 'sell_type', 'profit', 'cash']
print(f'{trades_df.profit.sum():0.2f}')
trades_df


1146.34


,trade_id,buy_date,buy_price,stop_loss,target_price,amount_invested,shares,buy_amount,sell_date,sell_price,shares,sell_amount,sell_type,profit,cash
0,0,2018-02-14 00:00:00-05:00,21.389334,19.250400,27.806134,5000.000000,233,4983.714758,2018-02-20 00:00:00-05:00,22.318001,233,5200.094185,Close_,216.379427,10200.094185
1,1,2018-03-22 00:00:00-04:00,20.926001,18.833401,27.203801,5100.047092,243,5085.018145,2018-03-27 00:00:00-04:00,18.612000,243,4522.715881,Close_,-562.302263,9622.762974
2,2,2018-04-02 00:00:00-04:00,17.084000,15.375600,22.209200,4811.381487,281,4800.603897,2018-04-09 00:00:00-04:00,19.310667,281,5426.297438,Close_,625.693541,10237.678925
3,3,2018-04-27 00:00:00-04:00,19.024668,17.122201,24.732068,5118.839462,269,5117.635622,2018-05-02 00:00:00-04:00,20.076668,269,5400.623634,Close_,282.988012,10519.463097
4,4,2018-05-17 00:00:00-04:00,19.059999,17.154000,24.777999,5259.731548,275,5241.499853,2018-05-22 00:00:00-04:00,18.334000,275,5041.849899,Close_,-199.649954,10301.581448
5,5,2018-05-29 00:00:00-04:00,18.567333,16.710600,24.137533,5150.790724,277,5143.151302,2018-06-04 00:00:00-04:00,19.782667,277,5479.798803,Close_,336.647501,10630.589527
6,6,2018-07-18 00:00:00-04:00,21.666668,19.500001,28.166668,5315.294764,245,5308.333645,2018-07-23 00:00:00-04:00,20.213333,245,4952.266617,Close_,-356.067028,10267.561380
7,7,2018-07-26 00:00:00-04:00,20.323334,18.291000,26.420334,5133.780690,252,5121.480103,2018-07-31 00:00:00-04:00,19.875999,252,5008.751862,Close_,-112.728241,10142.532552
8,8,2018-09-20 00:00:00-04:00,20.237333,18.213600,26.308533,5071.266276,250,5059.333324,2018-09-25 00:00:00-04:00,20.066000,250,5016.499996,Close_,-42.833328,10087.766272
9,9,2018-10-29 00:00:00-04:00,22.497999,20.248199,29.247399,5043.883136,224,5039.551819,2018-11-05 00:00:00-05:00,22.760000,224,5098.240051,Close_,58.688232,10142.123187
